<a href="https://colab.research.google.com/github/KOMBOU12/Marius/blob/main/Active_Space_Robustness_ResNetTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import math
import pickle
import random
from google.colab import drive
import time
from zipfile import ZipFile

In [ ]:
drive.mount('/content/gdrive') ### connexion au drive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
file_name = "/content/gdrive/MyDrive/Entrainement_05.zip" ### il faut créer un fichier data_images dans le drive et y déposer le fichier zip Entrainement.zip
with ZipFile(file_name,'r') as zip:                                ### cette partie du code dézip le fichier et le rend accessible dans le colab
  zip.extractall()
  print("Done")

Done


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  ### définition de device si GPU disponibles et fonction pour tranformer les images

mean  = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

data_transforms = {
    'train' : transforms.Compose([
       transforms.RandomResizedCrop(224),
       transforms.RandomHorizontalFlip(),
       transforms.ToTensor(),
       transforms.Normalize(mean,std)
    ]),
    'val': transforms.Compose([
       transforms.Resize(256),
       transforms.CenterCrop(224),
       transforms.ToTensor(),
       transforms.Normalize(mean,std)
    ])
}

In [ ]:
### Partie du code qui définit les fonctions d'entrainement et de test

data_dir = "/content/Entrainement_05"
sets = ['train', 'val']

num_clas = 7

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir,x),data_transforms[x]) for x in sets}
dataloaders = {x:torch.utils.data.DataLoader(image_datasets[x], batch_size = 64, shuffle=True, num_workers = 0) for x in sets}

#dataloaders = {x:torch.utils.data.DataLoader(data_augmented[x], batch_size = 64, shuffle=True, num_workers = 0) for x in sets}


datasets_size = {x: len(image_datasets[x]) for x in sets}
#datasets_size = {x: len(data_augmented[x]) for x in sets}
classe_names = image_datasets['train'].classes

print(classe_names)


def train_model(model, criterion, optimizer, scheduler, num_epochs = 20):
  since = time.time()
  best_model_wts = copy.deepcopy(model.state_dict())
  best_acc = 0.0

  for epoch in range(num_epochs):
    print(f'Epoque {epoch}/{num_epochs -1}')
    print('-'*20)

    for phase in ['train', 'val']:
      if phase == 'train':
        model.train()
      else:
        model.eval()
      running_loss = 0.0
      running_corrects = 0

      for inputs, labels in dataloaders[phase]:
        inputs = inputs.to(device)
        labels = labels.to(device)

        with torch.set_grad_enabled(phase == 'train'):
          outputs = model(inputs)
          _, preds = torch.max(outputs, 1)
          loss = criterion(outputs, labels)

          if phase == 'train':
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)


      if phase == 'train':
        scheduler.step()

      epoch_loss = running_loss / datasets_size[phase]
      epoch_acc  = running_corrects.double() / datasets_size[phase]

      print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

      if phase == 'val' and epoch_acc > best_acc:
        best_acc = epoch_acc
        best_model_wts = copy.deepcopy(model.state_dict())
    print()

  time_elapsed = time.time() - since
  print(f'Entraînement complété en {time_elapsed//60:.0f}m {time_elapsed%60:.0f}s')
  print(f'Best val Acc: {best_acc:.4f}')

  model.load_state_dict(best_model_wts)

  return model


def test_model(model,criterion):
  best_acc = 0.0
  num_epochs = 1
  for epoch in range(num_epochs):
    print(f'Epoque {epoch}/{num_epochs -1}')
    print('-'*20)

    for phase in ['train', 'val']:
      if phase == 'train':
        model.train()
      else:
        model.eval()
      running_loss = 0.0
      running_corrects = 0

      for inputs, labels in dataloaders[phase]:
        inputs = inputs.to(device)
        labels = labels.to(device)

        with torch.set_grad_enabled(phase == 'train'):
          outputs = model(inputs)
          _, preds = torch.max(outputs, 1)
          loss = criterion(outputs, labels)



        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)




      epoch_loss = running_loss / datasets_size[phase]
      epoch_acc  = running_corrects.double() / datasets_size[phase]

      print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

      if phase == 'val' and epoch_acc > best_acc:
        best_acc = epoch_acc
        best_model_wts = copy.deepcopy(model.state_dict())
    print()


  print(f'Best val Acc: {best_acc:.4f}')


['BuggyCart', 'Camels', 'Dogs', 'Farms', 'Fruits', 'GoldenFish', 'Scorpions']


In [ ]:
data = []
for img, label in dataloaders['train']:
  for j in range(len(label)):
   data.append((torch.reshape(img[j],(3,224,224)).to(device),label[j].item()))
print(len(data))

1630


In [ ]:
testdata = []
for img, label in dataloaders['val']:
  for j in range(len(label)):
   testdata.append((torch.reshape(img[j],(3,224,224)).to(device),label[j].item()))
print(len(testdata))

345


In [ ]:
les_PDR = []
for a,b,c in les_points_de_rupture_ResNet50:
  les_PDR.append((torch.reshape(a[0],(3,224,224)).to(device),a[1]))
for elem in data:
  les_PDR.append(elem)

NameError: name 'les_points_de_rupture_ResNet50' is not defined

In [ ]:

data_augmented = {}
data_augmented['train'] = les_PDR
data_augmented['val'] = testdata


Modèles

In [ ]:
model = models.resnet50(pretrained = True)
num_ftrs = model.fc.in_features
for param in model.parameters():
  param.requires_grad = False

model.fc = nn.Linear(num_ftrs, 7)
model.to(device)
model50 = model

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 81.2MB/s]


In [ ]:
model = models.resnet101(pretrained = True)
num_ftrs = model.fc.in_features
for param in model.parameters():
  param.requires_grad = False

model.fc = nn.Linear(num_ftrs, 7)
model.to(device)
model101 = model

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:02<00:00, 79.2MB/s]


In [ ]:
model = models.resnet152(pretrained = True)
num_ftrs = model.fc.in_features
for param in model.parameters():
  param.requires_grad = False

model.fc = nn.Linear(num_ftrs, 7)
model.to(device)
model152 = model

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to /root/.cache/torch/hub/checkpoints/resnet152-394f9c45.pth
100%|██████████| 230M/230M [00:02<00:00, 86.0MB/s]


In [ ]:
criterion = nn.CrossEntropyLoss()                                                 ### fonction loss, optimizer et le scheduler pour adapter le learning_rat

optimizer50  = optim.SGD(model.parameters(), lr = 0.1)
#optimizer101 = optim.SGD(model101.parameters(), lr = 0.01)
#optimizer152 = optim.SGD(model152.parameters(), lr = 0.01)

step_lr_scheduler50  = lr_scheduler.StepLR(optimizer50, step_size = 8, gamma = 0.5)
#step_lr_scheduler101 = lr_scheduler.StepLR(optimizer101, step_size = 4, gamma = 0.5)
#step_lr_scheduler152 = lr_scheduler.StepLR(optimizer152, step_size = 4, gamma = 0.5)

Entraînement

ResNet18

In [ ]:
model = models.resnet18(pretrained=True)

num_ftrs = model.fc.in_features

model.fc = nn.Linear(num_ftrs, 7)
model.to(device)

model_save_name = 'ResNet18FinTune7.pt'
path = F"/content/gdrive/My Drive/{model_save_name}"
#model.load_state_dict(torch.load(path))
print("done")
model.to(device)

done


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
device  = 'cpu'

In [ ]:
fct = model152.to(device)
test_model(fct,criterion)

Epoque 0/0
--------------------
train Loss: 2.0041 Acc: 0.1755
val Loss: 1.9987 Acc: 0.1565

Best val Acc: 0.1565


ResNet50

In [ ]:
model50 = train_model(model50, criterion, optimizer50, step_lr_scheduler50, num_epochs = 16)   ### Entraînement

model_save_name = 'ResNet50FinTune7.pt'                                                        ### Sauvgarde du modèle sur le drive
path = F"/content/gdrive/My Drive/{model_save_name}"
torch.save(model50.state_dict(), path)
print("Done")

In [ ]:
model50 = models.resnet50()                                                                    ### Recupèration du modèle suavgardé

num_ftrs = model50.fc.in_features

model50.fc = nn.Linear(num_ftrs, 7)
model50.to(device)

model_save_name = 'ResNet50FinTune7.pt'
path = F"/content/gdrive/My Drive/{model_save_name}"
model50.load_state_dict(torch.load(path))
print("done")
model50.to(device)

ResNet50_05

In [ ]:
model50_05 = models.resnet50(pretrained=True)                                                                    ### Recupèration du modèle suavgardé

num_ftrs = model50_05.fc.in_features

model50_05.fc = nn.Linear(num_ftrs, 7)
model50_05.to(device)

In [ ]:
model50_05 = train_model(model50_05, criterion, optimizer50, step_lr_scheduler50, num_epochs = 16)   ### Entraînement

model_save_name = 'ResNet50_05FinTune7.pt'                                                        ### Sauvgarde du modèle sur le drive
path = F"/content/gdrive/My Drive/{model_save_name}"
torch.save(model50_05.state_dict(), path)
print("Done")

In [ ]:
model50_05 = models.resnet50()                                                                    ### Recupèration du modèle suavgardé

num_ftrs = model50_05.fc.in_features

model50_05.fc = nn.Linear(num_ftrs, 7)
model50_05.to(device)

model_save_name = 'ResNet50_05FinTune7.pt'
path = F"/content/gdrive/My Drive/{model_save_name}"
model50_05.load_state_dict(torch.load(path))
print("done")
model50_05.to(device)

ResNet50 post points de ruptures

In [ ]:
model50PDR = models.resnet50(pretrained = True)                                                                    ### Recupèration du modèle suavgardé

num_ftrs = model50PDR.fc.in_features

model50PDR.fc = nn.Linear(num_ftrs, 7)
model50PDR.to(device)



In [ ]:
#model50PDR = train_model(model50PDR, criterion, optimizer50, step_lr_scheduler50, num_epochs = 16)   ### Entraînement

model_save_name = 'ResNet50PDRFinTune7.pt'                                                        ### Sauvgarde du modèle sur le drive
path = F"/content/gdrive/My Drive/{model_save_name}"
torch.save(model50PDR.state_dict(), path)
print("Done")

In [ ]:
model50PDR = models.resnet50()                                                                    ### Recupèration du modèle suavgardé

num_ftrs = model50PDR.fc.in_features

model50PDR.fc = nn.Linear(num_ftrs, 7)
model50PDR.to(device)

model_save_name = 'ResNet50PDRFinTune7.pt'
path = F"/content/gdrive/My Drive/{model_save_name}"
model50PDR.load_state_dict(torch.load(path))
print("done")
model50PDR.to(device)

ResNet101

In [ ]:
model152 = train_model(model, criterion, optimizer152, step_lr_scheduler152, num_epochs = 16)  ### Entraînement


In [ ]:
model101 = train_model(model101, criterion, optimizer101, step_lr_scheduler101, num_epochs = 16)  ### Entraînement

model_save_name = 'ResNet101FinTune7.pt'                                                       ### Sauvgarde du modèle sur le drive
path = F"/content/gdrive/My Drive/{model_save_name}"
torch.save(model101.state_dict(), path)
print("Done")

In [ ]:
### cellule permetant de calculer les points de rupture

e = 0.001
fct = model152
les_points_de_rupture_ResNet101 = PointDeRupture(fct,data,e,p=2,version = -1)

ASMetric = []
EucDist = []

for j in range(7):
  ASMetric.append([])
  EucDist.append([])

for a,b,c in les_points_de_rupture_ResNet101:
  ASMetric[a[1]].append(c.item())
  EucDist[a[1]].append(b.item())

Gene_Graphe_colinéarités(ASMetric,"Stability Metric study of ResNet152")
Gene_Graphe_colinéarités(EucDist,"Euclidian distance between tipping point and the original point of ResNet152")


In [ ]:
device = 'cuda'

In [ ]:


test_model(model50_05,criterion)

In [ ]:
device = 'cpu'
model50 = model50.to(device)
test_model(model50,criterion)

In [ ]:
device = 'cpu'
model50 = model50.to(device)
test_model(model50,criterion)

In [ ]:
device = 'cpu'
model50PDR = model50PDR.to(device)
test_model(model50PDR,criterion)

In [ ]:
model101 = models.resnet101(pretrained = True)                                                                  ### Recupèration du modèle suavgardé

num_ftrs = model101.fc.in_features

model101.fc = nn.Linear(num_ftrs, 7)
model101.to(device)

model_save_name = 'ResNet101FinTune7.pt'
path = F"/content/gdrive/My Drive/{model_save_name}"
model101.load_state_dict(torch.load(path))
print("done")
model101.to(device)

In [ ]:
test_model(model101,criterion)

ResNet152

In [ ]:
model152 = train_model(model152, criterion, optimizer152, step_lr_scheduler152, num_epochs = 16)  ### Entraînement

model_save_name = 'ResNet152FinTune7.pt'                                                          ### Sauvgarde du modèle sur le drive
path = F"/content/gdrive/My Drive/{model_save_name}"
torch.save(model152.state_dict(), path)
print("Done")

In [ ]:
model152 = models.resnet152(pretrained = True)                                                                    ### Recupèration du modèle suavgardé

num_ftrs = model152.fc.in_features

model152.fc = nn.Linear(num_ftrs, 7)
model152.to(device)

model_save_name = 'ResNet152FinTune7.pt'
path = F"/content/gdrive/My Drive/{model_save_name}"
model152.load_state_dict(torch.load(path))
print("done")
model152.to(device)

stabilité

In [ ]:
### Cette partie du code n'est pas sensée être modifiée Il faut juste executer cette cellule pour que les fonctions soient définies

def Trajctoires(fct,elem,e, p, version):
  for param in fct.parameters():
    param.requires_grad = False
  print("Start")
  l = []
  u,q = elem
  u = torch.reshape(u,(1,3, 224, 224)).to(device)
  fct.eval()
  A = True
  with torch.no_grad():
      output = fct(u)
  l.append(output.cpu().numpy())
  while A:
    with torch.no_grad():
      output = fct(u)
    x = torch.autograd.functional.jacobian(fct,u)[0]
    x = UpDate( x, output, q, p, version)
    x = e*x
    x = torch.reshape(torch.from_numpy(x),(1,3,224,224))
    x = x.to(device)
    v = (u + x).float()
    with torch.no_grad():
      output_1 = fct(v)
      y_pred = output_1.argmax(1)
    u = v
    l.append(output_1.cpu().numpy())
    if y_pred.item()!= q:
      A = False
  return np.array(l)


def UnPoinDeRupture(fct,elem,e, p, version):
  for param in fct.parameters():
    param.requires_grad = False
  print("Start")

  u,q = elem
  u = torch.reshape(u,(1,3, 224, 224)).to(device)

  fct.eval()
  A = True
  val = 0
  while A:
    with torch.no_grad():
      output = fct(u)
    x = torch.autograd.functional.jacobian(fct,u)[0]
    x = UpDate( x, output, q, p, version)
    x = e*x
    x = torch.reshape(torch.from_numpy(x),(1,3,224,224))
    x = x.to(device)
    v = (u + x).float()
    val = metric1(v,val,e)
    with torch.no_grad():
      output_1 = fct(v)
      y_pred = output_1.argmax(1)
    u = v
    if y_pred.item()!= q:
      A = False

    #normm = torch.norm(input=v-data[1][0], p='fro', dim=None, keepdim=False, out=None, dtype=None)

  return ((v,q,elem[0]),norm(v-elem[0]),val)

def PointDeRupture(fct,data,e,p=2, version = -1):
  """ prend en paramètre
  -> une classifieur fct
  -> des données data
  -> un pas de déplacement e
  -> un flotant p ou 'inf' pour infinity
  -> la vérsion de la l'algorithme -1 prend en compte toutes les classes 1 prend en compte le premier adversaire, 2 prend en compte les deux plus forts...
  """
  since = time.time()
  res = []
  taille = len(data)
  j = 0
  for param in fct.parameters():
    param.requires_grad = False
  print("Start")
  for elem in data:
    u,q = elem
    u = torch.reshape(u,(1,3, 224, 224)).to(device)
    fct.eval()
    A = True
    val = 0
    while A:
      with torch.no_grad():
        output = fct(u)
      x = torch.autograd.functional.jacobian(fct,u)[0]
      x = UpDate( x, output, q, p, version)
      x = e*x
      x = torch.reshape(torch.from_numpy(x),(1,3,224,224))
      x = x.to(device)
      v = (u + x).float()
      val = metric1(v,val,e)
      with torch.no_grad():
        output_1 = fct(v)
        y_pred = output_1.argmax(1)
      u = v
      if y_pred.item()!= q:
        A = False

      #normm = torch.norm(input=v-data[1][0], p='fro', dim=None, keepdim=False, out=None, dtype=None)
    now = time.time()
    uptonow = round((now - since)/60)
    j+= 1
    ratio = round(j/taille,3)
    res.append(((v,q,elem[0]),Norm(v-elem[0],p),val))         ### ce qui est enregistré est (a,b,c) avec a contenant le point de rupture la classe à la quelle il est rataché ainsi que l'image de départ
    print("Pourcentage de travail avancé :" + str(ratio*100))       ### b est la norme p entre le point de départ et le point de rupture
    print("--"*15)                                                  ### c est la métrique de stabilité
    print("Temps de travail :" + str(uptonow))
    print("--"*15)
  print("Done")
  return res


def Version(output,q, version):
  """output est la sortie du nn, q est la classe qu'on étudie et version est un entier compris entre 1 et 6 ou égale à -1
  version correspond au nombre de classes prises en compte pour le calcule du chemin actif (-1) c'est toutes les classes
  et les autres valeurs de version c'est assez clair
  """
  if version == -1:
    output = output.cpu().numpy()
    output = SofTMax(output[0])
    output[q] *= -1
    return output

  elif version == 8:
    v = output.clone().cpu()
    output = output.cpu().numpy()
    v[0][q] = -100000
    mm, preds = torch.max(v,1)
    res = np.zeros(output[0].shape)
    res[q] = -0.5
    res[preds.item()] = 0.5
    return res

  elif version == 9:
    o = output.cpu().numpy()
    o = o[0]
    mm, preds = torch.min(output,1)
    if mm.item()<0:
      o -= mm.item()
    else:
      o += mm.item()
    o /= np.linalg.norm(o,1)
    o[q] *= -1
    return o

  else:
    l= np.zeros(output[0].shape)
    l[q] = 1
    v = output.clone().cpu()
    v[0][q] = -100000
    for j in range(version):
      mm, preds = torch.max(v,1)
      l[preds.item()] = 1
      v[0][preds] = -100000
    output = output.cpu().numpy()
    output = output[0]*np.array(l)
    res = 0
    ll = []

    for j in range(len(l)):
      c = np.exp(output[j])
      res += c
      output[j] = l[j]*c
    output[q] *= -1
    return output/res


def SofTMax(output):
  res = []
  sftm = 0
  for j in range(output.shape[0]):
    y = np.exp(output[j])
    res.append(y)
    sftm += y
  res = np.array(res)/(sftm + 10**(-12))
  return res


def UpDate(x,output,q,p,version):
  u = []
  output = Version(output,q,version)
  for j in range(len(x)):
    u.append(ReNorm(torch.reshape(x[j],(-1,)),p).cpu().numpy())
  u = np.array(u)
  u = np.dot(np.transpose(u),np.transpose(output))
  return u


def UpDate2(x,output,q):
  u = []
  output = output.cpu().numpy()
  output = SofTMax(output)
  output[0][q] = -output[0][q]
  for j in range(len(x)):
    w = torch.reshape(x[j],(1,3*224*224))
    mm, preds = torch.max(w,1)
    mm = mm.item()
    u.append((1/mm)*w.cpu().numpy())
  u = np.array(u)
  u = np.dot(np.transpose(u),np.transpose(output))
  return u





def visionEval(x,fct):
  pred_probab = fct(x)
  y_pred = pred_probab.argmax(1)
  print(f"Predicted class: {y_pred}")
  z = x.to('cpu')
  out = torchvision.utils.make_grid(z)
  imshow(out, title=None)

def imshow(inp, title=None):
    """Imshow for Tensor."""

    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)

def ReNorm_v1(u):
  m = torch.nn.functional.normalize(u, p=2, dim=0, eps=1e-12, out=None)
  return m


def ReNorm(u,p):
  m = Norm(u,p)
  if m == 0:
    m = 10**(-12)
  return u/m

def norm(u):
    return torch.norm(u)


def norm1(u):
    return torch.norm(u,p=1)

def norm_inf(u):
    w = torch.reshape(u,(1,3*224*224))
    mm, preds = torch.max(w,1)
    mm = mm.item()
    return mm


def Norm(u,p=2):
    if p == 'inf':
      w = torch.reshape(u,(1,3*224*224))
      mm, preds = torch.max(w,1)
      mm = mm.item()
      return mm
    else:
      return torch.norm(u,p=p)

def metric1(v,val,e):
    val = val + e*norm(v)
    return val

def metric2(v,val,e):
    val = val - e*norm(v)
    return val

In [ ]:
with torch.no_grad():
  pred = model152(data[0][0].reshape((1,3,224,224)))
print(pred)
q = pred.cpu().numpy()
print(q[0].shape)

In [ ]:

print(Version(pred,6, 9))
print(np.sum(Version(pred,6, 9)))

In [ ]:
print(SofTMax(pred.cpu().numpy()))

Calibrage du pas de déplacement

In [ ]:
### cette cellule trie l'ensemble des images selon les classes et tire aléatoirement n images par classe
n = 1

datasorted = [[],[],[],[],[],[],[]]
for x,y in data:
  datasorted[y].append((x,y))

precision_test_data = []
randsample = [[],[],[],[],[],[],[]]
for i in range(len(randsample)):
  randsample[i] = random.sample(range(len(datasorted[i])),n)

for j in range(7):
  for s in randsample[j]:
    precision_test_data.append(datasorted[j][s])




In [ ]:
### partie du code qui détermine le pas de déplacement dans l'algo des points de rupture
fct  = model
calibrage = []
precision = [0.05,0.01,0.005,0.001,0.0005,0.0001]
for e in precision:
  test_data = PointDeRupture(fct,precision_test_data,e,p=2,version = -1)
  res = 0
  for a,b,c in test_data:
    res+=c
  calibrage.append((res/len(test_data)).item())

### Affichage

fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(12, 8))
taille = len(precision)
x = np.linspace(1,taille,taille)
axs.scatter(x,calibrage,linewidths = 8)

plt.show()

In [ ]:
def colin(x,y):
  x,y = x.cpu(), y.cpu()
  x,y = x.reshape((-1,)), y.reshape((-1,))
  x,y = x/torch.norm(x,p=2), y/torch.norm(y,p=2)
  a = torch.matmul(x,y)
  return a.item()


def stat_test_versions(fct,data):
  l = [[],[],[]]
  for elem in data:
    l[0].append(numpisation(UnPoinDeRupture(fct,elem,0.01, 2, -1)[0]))
    l[1].append(numpisation(UnPoinDeRupture(fct,elem,0.01, 2, 8)[0]))
    l[2].append(numpisation(UnPoinDeRupture(fct,elem,0.01, 2, 9)[0]))
  return np.array(l)

def numpisation(elem):
  x,q,a = elem
  l= [x.cpu().numpy(),a.cpu().numpy()]
  return np.array(l)
### test ###


res = stat_test_versions(model,data)

In [ ]:
l=[]
for j in range(len(res[0])):
  x,a = res[0][j]
  y,b = res[1][j]
  z,c = res[2][j]
  x = torch.from_numpy(x-a)
  y = torch.from_numpy(y-b)
  z = torch.from_numpy(z-c)
  l.append((colin(x,y),colin(z,y),colin(x,z)))
l = np.array(l)/len(res[0])

print(np.sum(l,axis=0))


In [ ]:
np.sum(l,axis=0)

In [ ]:
### partie du code qui détermine le pas de déplacement dans l'algo des points de rupture
fct  = model152
calibrage = []
precision = [0.05,0.01,0.005,0.001]#,0.0005,0.0001]
for e in precision:
  test_data = PointDeRupture(fct,precision_test_data,e,p=2,version = 1)
  res = 0
  for a,b,c in test_data:
    res+=c
  calibrage.append((res/len(test_data)).item())

### Affichage

fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(12, 8))
taille = len(precision)
x = np.linspace(1,taille,taille)
axs.scatter(x,calibrage,linewidths = 8)

plt.show()

In [ ]:
device


In [ ]:
### permet de créer un ensemble d'images pour faire tourner l'algo des points de rupture
#data = []
#for img, label in dataloaders['train']:
#  for j in range(len(label)):
#    data.append((torch.reshape(img[j],(3,224,224)).to(device),label[j].item()))

### cellule permetant de calculer les points de rupture

e = 0.1
fct = model50_05.to(device)

#t = time.time()
#les_points_de_rupture_ResNet50pdr = PointDeRupture(fct,data,e,p=2,version = 8)
#s = time.time()

tt = time.time()
les_points_de_rupture_ResNet50 = PointDeRupture(fct,data,e,p=2,version = 8)
ss = time.time()

#nom = str(les_points_de_rupture_ResNet50)
#path = '/content/gdrive/MyDrive/' + nom + '.pickle'

#with open(path, 'wb') as f_lpr:
#   pickle.dump(les_points_de_rupture_ResNet50, f_lpr)


#ASMetric = []
#EucDist = []

#for j in range(7):
#  ASMetric.append([])
#  EucDist.append([])

#for a,b,c in les_points_de_rupture_ResNet50pdr:
#  ASMetric[a[1]].append(c.item())
#  EucDist[a[1]].append(b.item())

#Gene_Graphe_colinéarités(ASMetric,"Stability metric study of ResNet50PDR")

#Gene_Graphe_colinéarités(EucDist,"Euclidean distance between tipping point and the original point of ResNet50PDR")

ASMetric = []
EucDist = []

for j in range(7):
  ASMetric.append([])
  EucDist.append([])

for a,b,c in les_points_de_rupture_ResNet50:
  ASMetric[a[1]].append(c.item())
  EucDist[a[1]].append(b.item())

Gene_Graphe_colinéarités(ASMetric,"Stability metric study of ResNet50_05")

Gene_Graphe_colinéarités(EucDist,"Euclidean distance between tipping point and the original point of ResNet50_05")
#print('Le temps :' + str(s-t))
print('Le temps :' + str(ss-tt))

In [ ]:
Gene_Graphe_colinéarités(ASMetric,"Stability metric study of ResNet50_05")

Gene_Graphe_colinéarités(EucDist,"Euclidean distance between tipping point and the original point of ResNet50_05")
#print('Le temps :' + str(s-t))

In [ ]:
with open('/content/gdrive/MyDrive/data.pickle', 'wb') as f_w:
    pickle.dump(les_points_de_rupture_ResNet50t_pdr, f_w)


In [ ]:
with open('/content/gdrive/MyDrive/data.pickle', 'rb') as f_r:
    les_points_de_rupture_ResNet50t_pdr = pickle.load(f_r)

In [ ]:
Gene_Graphe_colinéarités(ASMetric_t,"__")

Gene_Graphe_colinéarités(EucDist_t,"___")

Graphiques

In [ ]:
def Gene_Graphe_colinéarités(las,nom_du_graphique):

  fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(19, 8))

  # generate some random test data
  all_data = las

  # plot box plot
  axs.boxplot(all_data)
  axs.set_title(nom_du_graphique)


  axs.yaxis.grid(True)

  axs.set_xlabel("")
  axs.set_ylabel('Observed values')

  # add x-tick labels


  plt.setp(axs, xticks=[y + 1 for y in range(len(all_data))],
          xticklabels = classe_names)

  plt.savefig(nom_du_graphique+'.pdf')

  plt.show()

In [ ]:
### Cette cellule recupère l'information fournie par l'algo des points de rupture

ASMetric = []
EucDist = []

for j in range(7):
  ASMetric.append([])
  EucDist.append([])

for a,b,c in les_points_de_rupture_ResNet50:
  ASMetric[a[1]].append(c.item())
  EucDist[a[1]].append(b.item())

In [ ]:
ASM = [0,0,0,0,0,0,0]
Euc = [0,0,0,0,0,0,0]

for a,b,c in les_points_de_rupture_ResNet50:
  ASM[a[1]] += c.item()
  Euc[a[1]] += b.item()

ASM = np.array(ASM)/len(les_points_de_rupture_ResNet50)
Euc = np.array(Euc)/len(les_points_de_rupture_ResNet50)


In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(12, 8))
taille = 7
x = np.linspace(1,taille,taille)
axs.scatter(x, ASM,linewidths = 8, label = 'ASM distance')
axs.scatter(x, Euc,linewidths = 8, label = 'Euclidean distance')

plt.legend()
plt.show()

In [ ]:
Gene_Graphe_colinéarités(ASMetric,"Stability Metric study of ResNet152")

In [ ]:
Gene_Graphe_colinéarités(EucDist,"Euclidian distance between tipping point and the original point of ResNet152")

D'autres graphes

In [ ]:
with torch.no_grad():
    u = torch.clone(precision_test_data[0][0])
    pred = model152(torch.reshape(u,(1,3, 224, 224)).to(device))
print(pred)

In [ ]:
fct  = model152
e = 0.01

res7 = Trajctoires(fct,precision_test_data[0],e,2, -1)
print('fait')
res0 = Trajctoires(fct,precision_test_data[0],e,2, 0)
print('fait')
res1 = Trajctoires(fct,precision_test_data[0],e,2, 1)
print('fait')
res2 = Trajctoires(fct,precision_test_data[0],e,2, 2)
print('fait')
res3 = Trajctoires(fct,precision_test_data[0],e,2, 3)
print('fait')
res4 = Trajctoires(fct,precision_test_data[0],e,2, 4)
print('fait')
res8 = Trajctoires(fct,precision_test_data[0],e,2, 8)
print('fait')
res9 = Trajctoires(fct,precision_test_data[0],e,2, 9)
print('fait')


In [ ]:
figure, axs  = plt.subplots(2,4, figsize = (20,10))

axs[0,0].plot(res7[:,0,:])

axs[0,1].plot(res0[:,0,:])

axs[0,2].plot(res1[:,0,:])

axs[0,3].plot(res2[:,0,:])

axs[1,0].plot(res3[:,0,:])

axs[1,1].plot(res4[:,0,:])

axs[1,2].plot(res8[:,0,:])

axs[1,3].plot(res9[:,0,:])


plt.show()

In [ ]:
res8 = Trajctoires(fct,precision_test_data[0],e,2, 8)
print('fait')
figure, axs  = plt.subplots(1,1, figsize = (5,3))

axs.plot(res[:,0,:])


plt.show()

In [ ]:
#res9 = Trajctoires(fct,precision_test_data[0],e,2, 8)
print('fait')
figure, axs  = plt.subplots(1,1, figsize = (5,3))

axs.plot(res9[:,0,:])


plt.show()

In [ ]:
print(res7[-1])
print("##"*25)
print(res1[-1])
print("##"*25)
print(res2[-1])
print("##"*25)
print(res3[-1])
print("##"*25)
print(res4[-1])
print("##"*25)
print(res5[-1])

In [ ]:
classif2= [[],[],[],[],[],[],[]]
for a,b,c in test_data2:
  with torch.no_grad():
    u = torch.clone(a[-1])
    v = torch.clone(a[0])
    pred = model152(torch.reshape(u,(1,3, 224, 224)).to(device))
    fred = model152(torch.reshape(v,(1,3, 224, 224)).to(device))
    res = torch.argmax(pred).item()
    ser = torch.argmax(fred).item()
    classif2[res].append((res,ser))
    print("ASM")
    print(c)
    print("Euc")
    print(b)


In [ ]:
### cette cellule trie l'ensemble des images selon les classes et tire aléatoirement n images par classe
n = 1

datasorted = [[],[],[],[],[],[],[]]
for x,y in data:
  datasorted[y].append((x,y))

precision_test_data = []
randsample = [[],[],[],[],[],[],[]]
for i in range(len(randsample)):
  randsample[i] = random.sample(range(len(datasorted[i])),n)

for j in range(7):
  for s in randsample[j]:
    precision_test_data.append(datasorted[j][s])

In [ ]:
def Aug(x,a):
  x = x.cpu().reshape((3,224,224))
  x = np.abs(x.numpy())
  x = torchvision.utils.make_grid(torch.from_numpy(np.exp(a*x) -1))
  x = x.numpy().transpose((1, 2, 0))
  x = std * x + mean
  x = np.clip(x, 0, 1)

  return x


fct  = model152
e = 0.01
grid_data2 = [[],[],[],[],[],[],[]]

test_data2 = []
for k in range(7):
  a,b = precision_test_data[k]
  a = a.to(device)
  a = a.float()
  test_data2.append(UnPoinDeRupture(fct,(a,b),e,2, 8))
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])


for a,b,c in test_data2:
  v,w,z = a
  #z = z.cpu().reshape((3,224,224))
  s = torchvision.utils.make_grid(z.cpu())
  s = s.numpy().transpose((1, 2, 0))
  s = std * s + mean
  s = np.clip(s, 0, 1)

  #v = v.cpu().reshape((3,224,224))
  t = torchvision.utils.make_grid(v.cpu())
  t = t.numpy().transpose((1, 2, 0))
  t = std * t + mean
  t = np.clip(t, 0, 1)

  h = (z-v)
  #h = h.cpu().reshape((3,224,224))
  r = torchvision.utils.make_grid(h.cpu())
  r = r.numpy().transpose((1, 2, 0))
  r = std * r + mean
  r = np.clip(r, 0, 1)

  grid_data2[w] = (s,t,r,Aug(h,1000))

In [ ]:
classif2= [[],[],[],[],[],[],[]]
for a,b,c in test_data2:
  with torch.no_grad():
    u = torch.clone(a[-1])
    v = torch.clone(a[0])
    pred = model152(torch.reshape(u,(1,3, 224, 224)).to(device))
    fred = model152(torch.reshape(v,(1,3, 224, 224)).to(device))
    res = torch.argmax(pred).item()
    ser = torch.argmax(fred).item()
    classif2[res].append((res,ser))
    #print("ASM")
    #print(c)
    #print("Euc")
    #print(b)

In [ ]:
for j in range(7):
  print(classif2[j])

In [ ]:
from matplotlib import cbook
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid


imgs = grid_data2



def demo_grid():

    fig, axs = plt.subplots(4,7,figsize=(21,12),sharey=True)

    l = ["A","B","B-A","exp(a*(B-A)) -1"]

    for i in range(4):
      for k in range(7):

        axs[i,k].imshow(grid_data2[k][i])
        #axs[i,k].set_axis_off()
        axs[i,k].set(xticks=[], yticks=[])
        if i ==0:
          axs[i, k].set(xlabel='A'+' classified as '+str(classe_names[classif2[k][0][0]]))
        elif i ==1:
          axs[i, k].set(xlabel='B'+' classified as '+str(classe_names[classif2[k][0][1]]))
        elif i ==2:
          axs[i, k].set(xlabel='B-A')
        elif i ==3:
          axs[i, k].set(xlabel='Saturated (B-A)')


#fig = plt.figure(figsize=(120, 80))

demo_grid()
plt.savefig("Grille de comapraison saturation exp(abs) ResNet152")
plt.show()

In [ ]:

demo_grid()
plt.savefig("Grille de comapraison saturation exp(abs) ResNet152")
plt.show()